In [1]:
# This ensures that a gpu is being used by the current google colab session.
# If testing ES, then this block ensures that it is not available

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Oct  2 20:28:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# This code block is used to access your google drive

from google.colab import drive
ROOT = "/content/drive"
drive.mount(ROOT)

Mounted at /content/drive


In [3]:
# Make sure this points to the project folder

%cd drive/'My Drive'/CORL

/content/drive/My Drive/CORL


In [4]:
%cd l2i

/content/drive/My Drive/CORL/l2i


In [5]:
# import projects
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.insert(0,"../attention/")

class Config:

    def __init__(self):
        self.problem_seed = 1
        self.test_model = None
        self.num_training_points = 100
config = Config()

import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from generate_data import generate_vrp_data
from utils import load_model
from problems import CVRP
from problem import generate_problem
import pickle

device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
# make key for each problem
def make_key(problem):
  demand = tuple(np.array(problem.capacities[1:])/np.array(problem.capacities[0]))
  depot_loc = tuple(problem.locations[0][:2])
  locations = tuple(map(tuple, problem.locations[1:,:2]))
  return (demand, depot_loc, locations)

# convert l2i problem to attention problem
def convert_to_attention(problem):
  demand = np.array(problem.capacities[1:])/np.array(problem.capacities[0])
  depot_loc = problem.locations[0][:2]
  locations = problem.locations[1:,:2]

  dataset = CVRP.make_dataset(size=100, num_samples=1)
  dataset.data = [
                  {
                      'loc' : torch.tensor(locations, dtype=torch.float32, device=torch.device(device)),
                      'demand' : torch.tensor(demand, dtype=torch.float32, device=torch.device(device)),
                      'depot' : torch.tensor(depot_loc, dtype=torch.float32, device=torch.device(device))
                  }
  ]
  return dataset

# convert attention solution to l2i solution
def convert_to_l2i(solution):
  solist = solution.tolist()
  converted_solution = []
  route = [0]
  for ele in solist:
    route.append(ele)
    if not ele:
      converted_solution.append(route)
      route = [0]
  route.append(0)
  converted_solution.append(route)
  converted_solution.append([0,0])
  return converted_solution

def use_attention(problem):
  # convert to attention problem
  att_prob = convert_to_attention(problem)

  # Need a dataloader to batch instances
  dataloader = DataLoader(att_prob, batch_size=1)

  # Make var works for dicts
  batch = next(iter(dataloader))

  # Run the model
  model.eval()
  model.set_decode_type('greedy')
  with torch.no_grad():
    length, log_p, pi = model(batch, return_pi=True)

  # convert to l2i solution
  solution = convert_to_l2i(pi[0])

  return solution

In [ ]:
# This block will load the attention-based model and save the solutions
# change the load and save locations as needed.

load_ = "../models/att/21hr-model.pt"
save_ = "../init_sols/att_21_init_sol.pickle"

# load model
model = torch.load(load_, map_location=torch.device(device))

# Generate problem solution memory and set seed
N_ = 200 # samples
memory_ = {}
config.problem_seed = 1

solutions = []
for _ in range(N_):
    problem = generate_problem(config)
    solutions.append(use_attention(problem))
pickle.dump(solutions, open(save_, "wb"))